## Libararies

In [ ]:
import os
import pandas as pd

## Load Dataset

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("wordsforthewise/lending-club")

print("Path to dataset files:", path)


Path to dataset files: /kaggle/input/lending-club


In [3]:
# Use correct file path and filename
filepath = os.path.join(path, "accepted_2007_to_2018Q4.csv.gz")

df = pd.read_csv(filepath,low_memory=False)
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


## Filter out relevent columns for our research question

In [4]:
relevant_columns = [
    "loan_amnt",
    "term",
    "int_rate",
    "installment",
    "grade",
    "sub_grade",
    "emp_length",
    "home_ownership",
    "annual_inc",
    "verification_status",
    "purpose",
    "dti",
    "delinq_2yrs",
    "earliest_cr_line",
    "fico_range_low",
    "fico_range_high",
    "inq_last_6mths",
    "mths_since_last_delinq",
    "mths_since_last_record",
    "open_acc",
    "pub_rec",
    "revol_bal",
    "revol_util",
    "total_acc",
    "initial_list_status",
    "application_type",
    "mort_acc",
    "pub_rec_bankruptcies",
    "loan_status",
]
df = df[relevant_columns]
df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,...,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,loan_status
0,3600.0,36 months,13.99,123.03,C,C4,10+ years,MORTGAGE,55000.0,Not Verified,...,7.0,0.0,2765.0,29.7,13.0,w,Individual,1.0,0.0,Fully Paid
1,24700.0,36 months,11.99,820.28,C,C1,10+ years,MORTGAGE,65000.0,Not Verified,...,22.0,0.0,21470.0,19.2,38.0,w,Individual,4.0,0.0,Fully Paid
2,20000.0,60 months,10.78,432.66,B,B4,10+ years,MORTGAGE,63000.0,Not Verified,...,6.0,0.0,7869.0,56.2,18.0,w,Joint App,5.0,0.0,Fully Paid
3,35000.0,60 months,14.85,829.90,C,C5,10+ years,MORTGAGE,110000.0,Source Verified,...,13.0,0.0,7802.0,11.6,17.0,w,Individual,1.0,0.0,Current
4,10400.0,60 months,22.45,289.91,F,F1,3 years,MORTGAGE,104433.0,Source Verified,...,12.0,0.0,21929.0,64.5,35.0,w,Individual,6.0,0.0,Fully Paid


## Target Variables

In [5]:
df['loan_status'].value_counts()

,count
loan_status,
Fully Paid,1076751
Current,878317
Charged Off,268559
Late (31-120 days),21467
In Grace Period,8436
Late (16-30 days),4349
Does not meet the credit policy. Status:Fully Paid,1988
Does not meet the credit policy. Status:Charged Off,761
Default,40


**Target feature change into Binary**

In [6]:
# Define the mapping
loan_status_mapping = {
    "Fully Paid": 1,
    "Current": 1,
    "In Grace Period": 1,
    "Does not meet the credit policy. Status:Fully Paid": 1,
    "Late (31-120 days)": 0,
    "Charged Off": 0,
    "Default": 0,
    "Does not meet the credit policy. Status:Charged Off": 0,
}

# Apply the mapping to the 'Loan_Status' column
df['loan_status_binary'] = df['loan_status'].map(loan_status_mapping)
df.drop('loan_status', axis=1, inplace=True)

df['loan_status_binary'].value_counts()


,count
loan_status_binary,
1.0,1965492
0.0,290827


## Missing Values

In [7]:
def missing_data_summary(df, threshold=0):
    """
    Summarizes missing data, showing count and percentage of missing values for each column.
    Filters columns based on a missing percentage threshold.

    Parameters:
        df (pd.DataFrame): The dataframe to analyze.
        threshold (float): The minimum percentage of missing data to include in the summary.

    Returns:
        pd.DataFrame: A summary of missing data.
    """
    return (pd.DataFrame(df.isna().sum())
            .reset_index()
            .rename(columns={'index': 'Column', 0: 'mis_count'})
            .query('mis_count > 0')  # Only include columns with missing values
            .assign(Missing_Percentage=lambda x: x['mis_count'] / df.shape[0] * 100)
            .query(f'Missing_Percentage > {threshold}')  # Filter by threshold
            .sort_values('mis_count', ascending=False)
            .reset_index(drop=True))

missing = missing_data_summary(df,51)
missing

,Column,mis_count,Missing_Percentage
0,mths_since_last_record,1901545,84.113069
1,mths_since_last_delinq,1158535,51.246715


In [8]:
# Convert the first column of 'missing' to a list
columns_to_drop = missing.iloc[:, 0].tolist()

# Drop these columns from the DataFrame 'df'
df = df.drop(columns=columns_to_drop, errors='ignore')

df.isna().sum()

,0
loan_amnt,33
term,33
int_rate,33
installment,33
grade,33
sub_grade,33
emp_length,146940
home_ownership,33
annual_inc,37
verification_status,33


In [9]:
# fill missing values with 'mode' and 'mean'

# Columns with numerical data that can be filled with the mean
numerical_cols_to_fill_mean = [
    'loan_amnt',
    'int_rate',
    'installment',
    'annual_inc',
    'dti',
    'delinq_2yrs',
    'fico_range_low',
    'fico_range_high',
    'inq_last_6mths',
    'open_acc',
    'pub_rec',
    'revol_bal',
    'revol_util',
    'total_acc',
    'pub_rec_bankruptcies',
    'mort_acc'
]

# Fill missing values in numerical columns with the mean
for col in numerical_cols_to_fill_mean:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mean())

# Columns with categorical data that can be filled with the mode
categorical_cols_to_fill_mode = [
    'term',
    'grade',
    'sub_grade',
    'emp_length',
    'home_ownership',
    'earliest_cr_line',
    'verification_status',
    'purpose',
    'initial_list_status',
    'application_type'
]

# Fill missing values in categorical columns with the mode
for col in categorical_cols_to_fill_mode:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mode()[0])

# Final drop: rows where 'loan_status_binary' is missing
df.dropna(subset=['loan_status_binary'], inplace=True)

# Check for remaining missing values
print("\nMissing values after filling:")
print(df.isna().sum())


Missing values after filling:
loan_amnt               0
term                    0
int_rate                0
installment             0
grade                   0
sub_grade               0
emp_length              0
home_ownership          0
annual_inc              0
verification_status     0
purpose                 0
dti                     0
delinq_2yrs             0
earliest_cr_line        0
fico_range_low          0
fico_range_high         0
inq_last_6mths          0
open_acc                0
pub_rec                 0
revol_bal               0
revol_util              0
total_acc               0
initial_list_status     0
application_type        0
mort_acc                0
pub_rec_bankruptcies    0
loan_status_binary      0
dtype: int64


## Date time/Category Values

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2256319 entries, 0 to 2260698
Data columns (total 27 columns):
 #   Column                Dtype  
---  ------                -----  
 0   loan_amnt             float64
 1   term                  object 
 2   int_rate              float64
 3   installment           float64
 4   grade                 object 
 5   sub_grade             object 
 6   emp_length            object 
 7   home_ownership        object 
 8   annual_inc            float64
 9   verification_status   object 
 10  purpose               object 
 11  dti                   float64
 12  delinq_2yrs           float64
 13  earliest_cr_line      object 
 14  fico_range_low        float64
 15  fico_range_high       float64
 16  inq_last_6mths        float64
 17  open_acc              float64
 18  pub_rec               float64
 19  revol_bal             float64
 20  revol_util            float64
 21  total_acc             float64
 22  initial_list_status   object 
 23  application_

In [11]:
df.select_dtypes(include=['object']).head()

,term,grade,sub_grade,emp_length,home_ownership,verification_status,purpose,earliest_cr_line,initial_list_status,application_type
0,36 months,C,C4,10+ years,MORTGAGE,Not Verified,debt_consolidation,Aug-2003,w,Individual
1,36 months,C,C1,10+ years,MORTGAGE,Not Verified,small_business,Dec-1999,w,Individual
2,60 months,B,B4,10+ years,MORTGAGE,Not Verified,home_improvement,Aug-2000,w,Joint App
3,60 months,C,C5,10+ years,MORTGAGE,Source Verified,debt_consolidation,Sep-2008,w,Individual
4,60 months,F,F1,3 years,MORTGAGE,Source Verified,major_purchase,Jun-1998,w,Individual


In [15]:
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], format='%b-%Y')

In [21]:
# change 'earliest_crd_line' to 'crd_line_existed(months)' by different between 2018-12 and int type

from datetime import datetime

df['crd_line_existed(months)'] = ((datetime(2018, 12, 1).year - df['earliest_cr_line'].dt.year) * 12 +
                                  (datetime(2018, 12, 1).month - df['earliest_cr_line'].dt.month))

df.drop('earliest_cr_line', axis=1, inplace=True)

print(df[['crd_line_existed(months)']].head())
print(df['crd_line_existed(months)'].dtype)


   crd_line_existed(months)
0                       184
1                       228
2                       220
3                       123
4                       246
int64


**Strip leading/tailing white spaces**

In [12]:
df = df.apply(lambda col: col.str.strip() if col.dtypes == 'object' else col)

In [13]:
df['emp_length'].value_counts()

,count
emp_length,
10+ years,893177
2 years,203288
< 1 year,189561
3 years,180411
1 year,148116
5 years,139422
4 years,136351
6 years,102428
7 years,92544


**Stripping suffix and signs in 'term' and 'emp_length' column**

In [14]:
df['term'] = pd.to_numeric(df['term'].astype(str).str[:2].str.strip(), errors='coerce')
df['emp_length'] = pd.to_numeric(df['emp_length'].str.replace('<', '', regex=False).str[:2].str.strip(), errors='coerce')

**Convert dates to date-time type**

In [ ]:
def value_counter(df):
    for col in df.select_dtypes(["object"]).columns:
        print(df[col].value_counts(dropna = False))

value_counter(df)

grade
B    662564
C    648577
A    432686
D    323455
E    135237
F     41671
G     12129
Name: count, dtype: int64
home_ownership
mortgage    1109519
rent         893057
own          252512
other          1049
NaN             182
Name: count, dtype: int64
verification_status
Source Verified    884402
Not Verified       743609
Verified           628308
Name: count, dtype: int64
purpose
debt_consolidation    1275355
credit_card            516151
home_improvement       150132
other                  139130
major_purchase          50338
medical                 27439
small_business          24618
car                     23983
vacation                15499
moving                  15360
house                   14094
wedding                  2355
renewable_energy         1441
educational               424
Name: count, dtype: int64
initial_list_status
w    1532060
f     724259
Name: count, dtype: int64
application_type
Individual    2136115
Joint App      120204
Name: count, dtype: int64


In [17]:
# Suppose sub_grade is redundent as we have big group 'grade' columns
# in order to save the memory for encoding , I will drop
df.drop('sub_grade', axis=1, inplace = True)

In [18]:
# In home_ownership column, 'ANY,OTHER and NONE' do not make sense to be alone
# themselves so that will group into 'OTHER'
home_ownership_mapping = {
    'ANY' : 'other',
    'MORTGAGE' : 'mortgage',
    'RENT' : 'rent',
    'OWN' : 'own',
    'NONE' : 'other'
}
df.home_ownership = df.home_ownership.map(home_ownership_mapping)

## All to Numeric Features

In [24]:
cat_cols = df.select_dtypes(include=['object']).columns
num_cols = df.select_dtypes(exclude=['object']).drop('loan_status_binary',axis=1).columns

# Concatenate numeric columns explicitly
df_final = pd.concat([df[num_cols], pd.get_dummies(df[cat_cols], drop_first=True), df['loan_status_binary']], axis=1)
df_final.head()

,loan_amnt,term,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,...,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_w,application_type_Joint App,loan_status_binary
0,3600.0,36,13.99,123.03,10,55000.0,5.91,0.0,675.0,679.0,...,False,False,False,False,False,False,False,True,False,1.0
1,24700.0,36,11.99,820.28,10,65000.0,16.06,1.0,715.0,719.0,...,False,False,False,False,True,False,False,True,False,1.0
2,20000.0,60,10.78,432.66,10,63000.0,10.78,0.0,695.0,699.0,...,False,False,False,False,False,False,False,True,True,1.0
3,35000.0,60,14.85,829.90,10,110000.0,17.06,0.0,785.0,789.0,...,False,False,False,False,False,False,False,True,False,1.0
4,10400.0,60,22.45,289.91,3,104433.0,25.37,1.0,695.0,699.0,...,False,False,False,False,False,False,False,True,False,1.0


## Save Cleaned file

In [ ]:
# Save cleaned CSV for use in MS3

#df_final.to_csv("1_datasets/processed_data/p2p_df_final_cleaned.csv", index=False)

#print("✅ Cleaned file saved in /1_datasets/processed_datasets/")
